### 타이타닉 분석 및 시각화
- 데이터 :  titanic_train.csv
- (1) 생존자의 연령
- (2) 생존자의 성별
- (3) 생존자들의 연령과 성별 분석
- (4) 등석에 따른 생존자 분석

[1] 모듈 준비 및 데이터 로딩 <hr>

In [ ]:
## 모듈 로딩
import pandas as pd
import matplotlib.pyplot as plt
import koreanize_matplotlib

In [ ]:
## 데이터 준비
#DATA_FILE = '../Data/titanic.csv'
DATA_FILE = '../Data/titanic_train.csv'

## 데이터 로딩
dataDF = pd.read_csv(DATA_FILE)

## 복사본 준비
dataDF2 = dataDF.copy()

[2] 기본 데이터 확인 및 전처리 방향 설정 <hr>

In [ ]:
## -------------------------------------------------------------
## [2-1] 기본 데이터 확인 : head() / info() / describe()
## -------------------------------------------------------------
display( dataDF2.head() )

[3] 전처리 진행 <hr>